In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import json
from decimal import *
import time

In [ ]:
'''define bounding box. i'm using queensland as an example'''
North = -9.0000
East = 155.000
South = -29.500
West = 138.000

In [ ]:
# retains precision of coordinates by defining number of decimal places
getcontext().prec = 3

'''store each response from the api call in this list'''
list_of_requests = []

'''api key'''
API_KEY = <insert your Google API key here>

'''query as string. don't forget percent20 for spaces! i've used 'quarries queensland' as an example'''
q = 'quarries%20queensland'

'''function scans latitude points northwards along a fixed line of longitude until northern limit is reached.
a buffer of 50km radius is created around each point. 
relevant results within this zone are captured and added to above list (list_of_requests)'''
def scan(longitude,north,east,south,west,query):
    latitude = south
    pagetoken = ''
    while latitude < north:
        r = requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json?query='+query+'&location='+str(latitude)+','+str(longitude)+'&radius=50000&pagetoken='+pagetoken+'&key='+API_KEY)
        data = r.json()
        list_of_requests.append(data)
        try:
            pagetoken += data['next_page_token']
        except KeyError:
            latitude += 1
            pagetoken = ''
    
    return list_of_requests

'''function takes bounding box coordinates and api query as parameters. 
line of longitude slides incrementally east. 
each new line of longitude is scanned northwards in turn, as above, until eastern limit is reached'''
def increment_long(north,east,south,west,query):
    longitude = west
    while longitude < east:
        scan(longitude,north,east,south,west,query)
        longitude += 1
        longitude = Decimal(longitude)

increment_long(North,East,South,West,q)


In [ ]:
'''create pandas dataframe'''
df1 = pd.DataFrame(list_of_requests[0]['results'])
for i in list_of_requests:
    df2 = pd.DataFrame(i['results'])
    df1 = pd.concat([df1, df2], axis=1)

'''there will have been overlaps -- this drops duplicates'''
df1.drop_duplicates(subset='name', inplace=True)
df1 = df1.reset_index()


'''split out lat and long so they have their own columns'''
addresses = []
latlngs = []
for i in df1['Address']:
    addresses.append(i)

for i in addresses:
    g = geocoder.google(i, key=API_KEY)
    latlngs.append(g.latlng)

lats = []
lngs = []

for i in latlngs:
    try:
        lats.append(i[0])
        lngs.append(i[1])
    except TypeError:
        lats.append(None)
        lngs.append(None)
        
df1['Latitude'] = lats
df1['Longitude'] = lngs

'''create geopandas dataframe with new geometry column'''
gdf1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['Longitude'], df1['Latitude']))

'''set crs'''
def set_crs(geodataframe):
    geodataframe.crs = {'init':'epsg:4283'}
    geodataframe = geodataframe.to_crs(epsg=4283)
    return geodataframe
    
set_crs(gdf1)

'''delete invalid columns, such as those containing lists'''
del gdf1['photos'], gdf1['types']


'''convert to shapefile'''
gdf1.to_file('quarries_qld_google.shp')

